In [3]:
from pyspark import SparkContext

import sys
import re
import os

#import time
#t0 = time.time()
#sc = SparkContext(appName="Bla")

default_grpSize = 5
default_matchThr = 90
hashed = 1
#catalog_models = "" 
#use_sections = 0

def fillSchedule(billDirPath):
    scheduler = list()
    walkOpj =  os.walk(billDirPath)
    for (dirpath, dirnames, filenames) in walkOpj:
        for fname in filenames:
            if "catalog" not in fname: continue
            #print fname      
            #OK, that assumes the filename is catalog_SSYYYY, where S is state (2 letters), and Y is year (4 digits)
            info = fname.split("/")[-1].split("_")
            state = info[1][:2]
            year = info[1][2:]
            #try:
            #    billsSearched += int(open(dirpath+"/index_"+state+year).readline())
            #except:
            #    billsSearched += len(open(dirpath+"/"+fname).readlines())
            scheduler.append((state,year))
    return scheduler
           
def extractHashedSets(x,step,grpSize,hashed):
   wGrps = set()                 
   for n in range(0, len(x[1]), step):
      cgrouplst = x[1][n:n+grpSize]
      cgrouplst = filter(lambda x: len(x) > 1, cgrouplst)
      cgrp = " ".join(cgrouplst)
      if hashed:
         cgrp = hash(cgrp)
      wGrps.add(cgrp)

   return (x[0], wGrps)

def matchExtractor(x,model_size):
    matchCnt =len(x)
    matchPrecent = 0.
    matchPrecent = float(matchCnt)/ len(model_size) * 100.
    return matchPrecent   


def intersection_rdd(cart_rdd_element):
    sid = cart_rdd_element[1][0]
    mid = cart_rdd_element[0][0]
    s_wGrps = cart_rdd_element[1][1]
    m_wGrps = cart_rdd_element[0][1]
    model_size = len(m_wGrps)
    intersec = m_wGrps.intersection(s_wGrps)  
    matchCnt =len(intersec)
    matchPrecent = float(matchCnt)/ model_size * 100.
    return (mid, sid, matchPrecent)
    
   
#list of tuples (document id, document tokenized as list), have not stripped it
#for loop over all model bills - outer loop
prefix = '/Users/asvyatko/Desktop/Spark_tests/bills2/'
scheduler = fillSchedule(prefix) #[('CO','2009'),('NJ','2011'),('NJ','2012'),('NJ','2013'),('HI','2010')]
print scheduler
matches = list()
    
for state1,year1 in scheduler:
    model_folder = prefix+"catalog_"+state1+year1
    print "model_folder ", model_folder
    modelBills = sc.textFile(model_folder).map(lambda x: x.split('^^^'))\
               .map(lambda x: (x[0],x[1].encode("utf8").translate(None,'?<>,[]{}()*.0123456789:;-\'"_').lower()))\
               .map(lambda x: (x[0],x[1].split())).map(lambda x: extractHashedSets(x,1,default_grpSize,hashed))
    for state2,year2 in scheduler: 
        if state2 <= state1: continue 
        catalog_folder = prefix+"catalog_"+state2+year2              
        print "catalog_folder ", catalog_folder         
        catalogBills = sc.textFile(catalog_folder).map(lambda x: x.split('^^^'))\
               .map(lambda x: (x[0],x[1].encode("utf8").translate(None,'?<>,[]{}()*.0123456789:;-\'"_').lower()))\
               .map(lambda x: (x[0],x[1].split())).map(lambda x: extractHashedSets(x,1,default_grpSize,hashed))
        #extract a cartesian RDD
        cart_rdd = modelBills.cartesian(catalogBills)
        #print cart_rdd.count()
        match_rdd = cart_rdd.map(lambda x: intersection_rdd(x)).filter(lambda x: x[2] > default_matchThr)
        match_rdd.saveAsTextFile('./matches/'+state1+year1+"_"+state2+year2)
        #print match_rdd.take(5)
                
#end of code block
#t1 = time.time()
#print t1-t0
#print len(matches)  

[('AK', '1995'), ('AL', '2010')]
model_folder  /Users/asvyatko/Desktop/Spark_tests/bills2/catalog_AK1995
catalog_folder  /Users/asvyatko/Desktop/Spark_tests/bills2/catalog_AL2010


KeyboardInterrupt: 